In [7]:
import gymnasium as gym
from stable_baselines3 import PPO
import cv2
import numpy as np
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, VecTransposeImage
import os
import imageio


In [15]:
# Define the environment with preprocessing to match the training setup
def make_env():
    env = gym.make("CarRacing-v2", render_mode="rgb_array")
    env = Monitor(env)  # Wrap with Monitor to keep track of statistics
    return env

env = DummyVecEnv([make_env])
env = VecFrameStack(env, n_stack=4)  # Stack 4 frames
env = VecTransposeImage(env)  # Ensure correct format for image-based observations

# Test rendering capability
try:
    obs = env.reset()
    frame = env.render()
    if frame is None:
        raise ValueError("Environment rendering returned None. Ensure the environment supports rendering in 'rgb_array' mode.")
    print("Rendering is supported.")
except Exception as e:
    print(f"Error during rendering test: {e}")

steps = 1000000
Path = f'logs/ppo_carracing_model_{steps}_steps.zip'

# Initialize a new model with the correct observation and action spaces
model = PPO("CnnPolicy", env, verbose=1)

# Load the trained weights
model.set_parameters(Path)

# Function to capture and save video
def record_video(env, model, video_length=500, prefix=f'ppo_carracing_{steps}'):
    """
    Record a video of the agent using the trained model.
    """
    # Reset the environment and get the initial frame
    obs = env.reset()
    initial_frame = env.render()
    
    # Check if the initial frame is valid
    if initial_frame is None:
        raise ValueError("Environment rendering returned None. Ensure the environment supports rendering in 'rgb_array' mode.")

    frame_size = (initial_frame.shape[1], initial_frame.shape[0])
    video_filename = f"videos/{prefix}_video.avi"
    out = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'MJPG'), 20, frame_size)

    for _ in range(video_length):
        action, _states = model.predict(obs, deterministic=True)
        obs, rewards, dones, info = env.step(action)
        frame = env.render()
        
        # Check if frame is valid
        if frame is not None:
            out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        
        if dones:
            obs = env.reset()

    out.release()
    print(f"Video saved as {video_filename}")

Rendering is supported.
Using cpu device


In [16]:
# Record the video
record_video(env, model)

Video saved as videos/ppo_carracing_1000000_video.avi
